In [ ]:
import os
from itertools import groupby
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
import importlib
spec = importlib.util.spec_from_file_location("fspliter", "..\\utils\\files_spliter.py")
fspliter = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fspliter)

spec = importlib.util.spec_from_file_location("preprocessing", "..\\utils\\preprocessing.py")
preprocessing = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preprocessing)

In [ ]:
df_bread = fspliter.get_bread(0)
df_bread.shape

In [ ]:
df_mice = fspliter.get_mice(0)
df_mice.shape

## Filter by day

In [ ]:
df_day4 = fspliter.retrieve_day(df_bread[df_bread['filename'] == df_bread['filename'].unique()[0]], 3)
df_day4.shape

In [ ]:
df_day4 = fspliter.retrieve_day(df_mice, 3)
df_day4.shape

In [ ]:
df_test = fspliter.remove_day_from_bread(df_bread, 1)

In [ ]:
df_test.shape

# Visualisation

## Distribution of sleep-wake states

In [ ]:
print(df_bread['temp'].describe())

In [ ]:
df_mice = preprocessing.clean_missing_values(df_mice)
df_bread = preprocessing.clean_missing_values(df_bread)

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df_mice, x='state', order=df_mice['state'].value_counts().index)
# plt.title('Distribution of Sleep-Wake States')
# plt.xlabel('State')
# plt.ylabel('Count')
# plt.show()
stats = ["w", "n", "r"]

state_colors = {
    'w': '#113048',
    'n': '#D94C0C',
    'r': '#4D4B49',
}


import plotly.express as px

fig = px.histogram(df_mice, x='state', text_auto=True, color="state", color_discrete_map=state_colors).update_xaxes(categoryorder="array", categoryarray=stats)
fig.update_traces(textfont_size= 42, textposition='inside', textfont_color='#F0E2C8', insidetextanchor='middle')
fig.show()


Premièrement, on remarque que les états wake et non-REM sont les plus fréquents. Si on aditionne les états non-REM et REM, on peut constater que la souris a passé un peu plus de temps éveillée qu'endormie sur ces 4 jours. Cependant, pour le jour 3, la souris a été forcée de rester éveillée donc nous allons alors refaire l'expérience uniquement le jour 3.

In [ ]:
df_mice_day3 = fspliter.retrieve_day(df_mice, 3)
df_mice_day4 = fspliter.retrieve_day(df_mice, 4)

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df_mice_day3, x='state', order=df_mice_day3['state'].value_counts().index)
# plt.title('Distribution of Sleep-Wake States')
# plt.xlabel('State')
# plt.ylabel('Count')
# plt.show()


fig = px.histogram(df_mice_day3, x='state', text_auto=True, color="state", color_discrete_map=state_colors).update_xaxes(categoryorder="array", categoryarray=stats)
fig.update_traces(textfont_size= 42, textposition='inside', textfont_color='#F0E2C8', insidetextanchor='middle')
fig.show()


On remarque effectivement que le jour 3, la souris a passé une majorité de temps éveillée.

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df_mice_day4, x='state', order=df_mice_day4['state'].value_counts().index)
# plt.title('Distribution of Sleep-Wake States')
# plt.xlabel('State')
# plt.ylabel('Count')
# plt.show()


fig = px.histogram(df_mice_day4, x='state', text_auto=True, color="state", color_discrete_map=state_colors).update_xaxes(categoryorder="array", categoryarray=stats)
fig.update_traces(textfont_size= 42, textposition='inside', textfont_color='#F0E2C8', insidetextanchor='middle')
fig.show()

Ci-dessus, on remarque que le cycle de sommeil de la souris est revenu à la normal le jour 4 malgré le fait qu'elle ait été forcée de rester éveillée le jour 3.

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df_bread, x='state', order=df_bread['state'].value_counts().index)
# plt.title('Distribution of Sleep-Wake States')
# plt.xlabel('State')
# plt.ylabel('Count')
# plt.show()

fig = px.histogram(df_bread, x='state', text_auto=True, color="state", color_discrete_map=state_colors).update_xaxes(categoryorder="array", categoryarray=stats)
fig.update_traces(textfont_size= 42, textposition='inside', textfont_color='#F0E2C8', insidetextanchor='middle')
fig.show()

Pour toute la souche, on remarque que les souris passent légèrement plus de temps endormies qu'éveillées.

## Distribution of EEGv and EMGv features

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.boxplot(df_mice['EEGv'])
# plt.show()


fig = px.box(df_mice, x="EEGv", points = 'all')
fig.update_traces(marker_color='#113048', marker_line_color='#113048', marker_line_width=2, opacity=0.6)
fig.update_xaxes(range=[0, 0.00000004])
fig.show()


On remarque quelques valeurs aberrantes dans les données EEGv.

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.boxplot(df_mice['EMGv'])
# plt.show()

fig = px.box(df_mice, x="EMGv", points = 'all')
fig.update_traces(marker_color='#113048', marker_line_color='#113048', marker_line_width=2, opacity=0.6)
fig.update_xaxes(range=[0, 0.00000008])
fig.show()

On remarque quelques valeurs aberrantes dans les données EMGv.

In [ ]:
plt.figure(figsize=(14, 4))
sns.boxplot(x='state', y='EEGv', data=df_mice, showfliers=False)
plt.title('Boxplot of EEGv per State (without outliers)')
plt.show()

On remarque que la variance de l'activité électrique du cerveau est plus grande lorsque la souris dans l'état non-rem. L'activité est presque similaire quand la souris est éveillé ou dans l'état REM.

In [ ]:
plt.figure(figsize=(14, 5))
sns.boxplot(x='state', y='EMGv', data=df_mice, showfliers=False)
plt.title('Boxplot of EMGv per State (without outliers)')
plt.show()

On remarque que l'activité électrique des muscles est plus grande lorsque la souris est réveillée.

## Transitions between states

In [ ]:
def calculate_transition_between_states(df):

    states = sorted(list(set(df)))
    transition_mat = pd.DataFrame(0, index=states, columns=states, dtype=float)

    for (from_state, to_state) in zip(df[:-1], df[1:]):
        transition_mat.loc[from_state, to_state] += 1

    transition_mat = transition_mat.div(transition_mat.sum(axis=1), axis=0)
    return transition_mat

from matplotlib.colors import LinearSegmentedColormap

transitions = calculate_transition_between_states(df_bread['state'])
colors = ['#ffffff', '#113048']
cm = LinearSegmentedColormap.from_list(
  "Custom", colors, N=20)
plt.figure(figsize=(10, 8))
sns.heatmap(transitions, annot=True, fmt=".2f", cmap=cm)
# plt.title('State Transition Heatmap')
plt.xlabel('To State')
plt.ylabel('From State')
plt.show()


Premièrement, on peut voir sur ce graphique que quand la souris se trouve dans un état, elle a tendance à rester dans cet état. Par exemple, si la souris est dans l'état wake, elle a 0.94 de chance de rester dans cet état ce qui semble tout à fait logique. Ce qui est intéressant, c'est de voir que la souris aura comme le cycle de sommeil naturel l'indique, tendance a passé de l'état wake à l'état non-REM plutôt que l'état REM.

## Correlation between features

In [ ]:
correlation_matrix = df_mice.iloc[:, 4:-1].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap of the Features')
plt.show()


## Dimensionality reduction and visualization

In [ ]:
df_mice = preprocessing.remove_outliers(df_mice)

In [ ]:
sample_data = df_mice.sample(n=1000, random_state=42)
features = ['EEGv', 'EMGv'] + [f'bin{i}' for i in range(0, 401, 1)]
features_without_v = [f'bin{i}' for i in range(0, 401, 1)]

scaler = StandardScaler()
data_normalized = scaler.fit_transform(sample_data[features])

In [ ]:
umap = UMAP(n_neighbors=15, metric='cosine' ,min_dist=0.4, n_components=2)
data_umap = umap.fit_transform(data_normalized)
fig = px.scatter(data_umap, x=data_umap[:, 0], y=data_umap[:, 1], color=sample_data['state'], title='UMAP projection')
fig.show()

On remarque que les états wake et non-REM peuvent être assez bien séparés avec UMAP. Cependant, l'état REM semble être plus difficile à séparer des autres états.

In [ ]:
pca = PCA(n_components=50, random_state=42)
data_pca = pca.fit_transform(data_normalized)

tsne = TSNE(n_components=2, perplexity=50, n_iter=1500, random_state=42)
data_tsne = tsne.fit_transform(data_pca)

fig = px.scatter(data_tsne, x=data_tsne[:, 0], y=data_tsne[:, 1], color=sample_data['state'], title='T-sne projection with normalized data')
fig.show()

On remarque que les états wake et non-REM peuvent être assez bien séparés avec T-sne et en normalisant les données. Cependant, l'état REM semble être plus difficile à séparer des autres états.

In [ ]:
data_pca = pca.fit_transform(sample_data[features])

tsne = TSNE(n_components=2, perplexity=50, n_iter=1500)
data_tsne = tsne.fit_transform(data_pca)

fig = px.scatter(data_tsne, x=data_tsne[:, 0], y=data_tsne[:, 1], color=sample_data['state'], title='T-sne projection with non-normalized data')
fig.show()

En normalisant pas les données, on remarque que l'état REM est un peu plus prédictible que les autres méthodes vues précédemment mais cela reste difficile de séparer l'état REM des autres états.

In [ ]:
columns = ['EMGv', 'EEGv'] + [f'bin{i}' for i in range(401)]
data_pca = df_day4[columns]

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_pca)

max_components = min(len(columns), len(df_mice))
explained_variances = []
for n_components in range(1, max_components + 1, 5):
    pca = PCA(n_components=n_components)
    pca.fit(data_scaled)
    explained_variance = np.sum(pca.explained_variance_ratio_)
    explained_variances.append(explained_variance)

# Plotting the explained variance
plt.figure(figsize=(12, 6))
plt.plot(range(1, max_components + 1, 5), explained_variances, marker='o')
plt.title('Variance Expliquée par le Nombre de Composantes PCA')
plt.xlabel('Nombre de Composantes PCA')
plt.ylabel('Variance Expliquée (pourcentage)')
plt.grid(True)
plt.show()
